<a href="https://colab.research.google.com/github/ShuichiroTsuda/Kaggle_Liverpool_Ion_Switching/blob/master/FE1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/vbmokin/ion-switching-advanced-fe-lgb-xgb-confmatrix

In [0]:
import json
from google.colab import drive
import os
import pandas as pd

class ColabKaggle:

    base = 'gdrive/My Drive/'

    def __init__(self):
        drive.mount('gdrive')

    def load_api_key(self, path):
        """
        example of path: path = 'Colab Notebooks/Kaggle/API_setup/kaggle.json'

        """
        path = self.base + path
        with open(path) as f:
            kaggle_api_key = json.load(f)

        filename = "/root/.kaggle/kaggle.json"
        os.makedirs(os.path.dirname(filename), exist_ok=True)

        with open(filename, 'w') as f:
            json.dump(kaggle_api_key, f)

        os.chmod(filename, 600)

        print('API key loaded.')

    def save_df_as_pickle(self, df, path):
        """
        example of path: folder = 'Colab Notebooks/Kaggle/Liverpool_Ion_Switching/abc.pickle'

        """
        df.to_pickle(path)

    def pickle_to_df(self, path):
        return pd.read_pickle(path)

In [3]:
ck = ColabKaggle()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at gdrive


In [4]:
ck.load_api_key('Colab Notebooks/Kaggle/API_setup/kaggle.json')

API key loaded.


In [5]:
!kaggle competitions download -c liverpool-ion-switching

 94% 25.0M/26.6M [00:00<00:00, 33.5MB/s]
100% 26.6M/26.6M [00:00<00:00, 49.3MB/s]
 50% 5.00M/9.91M [00:00<00:00, 42.1MB/s]
100% 9.91M/9.91M [00:00<00:00, 63.3MB/s]
  0% 0.00/4.27M [00:00<?, ?B/s]
100% 4.27M/4.27M [00:00<00:00, 70.0MB/s]


In [6]:
!unzip sample_submission.csv.zip
!unzip test.csv.zip
!unzip train.csv.zip

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression, Ridge, SGDRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, f1_score, mean_absolute_error, make_scorer
import lightgbm as lgb

from functools import partial
import scipy as sp

import time
import datetime

import gc

In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        if col != 'time':
            col_type = df[col].dtypes
            if col_type in numerics:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [10]:
train.head()

,time,signal,open_channels
0,0.0001,-2.7600,0
1,0.0002,-2.8557,0
2,0.0003,-2.4074,0
3,0.0004,-3.1404,0
4,0.0005,-3.1525,0


In [11]:
train.shape

(5000000, 3)

In [12]:
test.head()

,time,signal
0,500.0001,-2.6498
1,500.0002,-2.8494
2,500.0003,-2.8600
3,500.0004,-2.4350
4,500.0005,-2.6155


In [13]:
test.shape

(2000000, 2)

In [0]:
seed_random = 42

In [15]:
# FE-2 & FE-3

# Seed for random_state
seed_random = 42

# FE-2 : window_sizes = [10, 25, 50, 100, 500, 1000, 5000, 10000, 25000, 50000, 100000, 500000]
# About selection of these values see my post: https://www.kaggle.com/c/liverpool-ion-switching/discussion/135073
window_sizes = [10, 50]
# with new features: signal_shift_+2, signal_shift_-2
# without ratio-, diff-, norm-features

# LGB model tuning
lr_lgb = 0.1
num_leaves = 200
num_iterations = 2000

# XGB model tuning
lr_xgb = 0.1
max_depth_xgb = 10
num_boost_round_xgb = 2000

# Set weight of models
w_lgb = 0.5
w_xgb = 1 - w_lgb
print(w_xgb)
# without LogReg, MLP, Ridge Regression and SGDRegressor models

0.5


In [16]:
%%time
for window in window_sizes:
    test["rolling_mean_" + str(window)] = train['signal'].rolling(window=window).mean()
    test["rolling_std_" + str(window)] = train['signal'].rolling(window=window).std()
    test["rolling_var_" + str(window)] = train['signal'].rolling(window=window).var()
    test["rolling_min_" + str(window)] = train['signal'].rolling(window=window).min()
    test["rolling_max_" + str(window)] = train['signal'].rolling(window=window).max()

test = test.replace([np.inf, -np.inf], np.nan)    
test.fillna(0, inplace=True)

CPU times: user 2.21 s, sys: 176 ms, total: 2.39 s
Wall time: 2.4 s


In [17]:
%%time
for window in window_sizes:
    train["rolling_mean_" + str(window)] = train['signal'].rolling(window=window).mean()
    train["rolling_std_" + str(window)] = train['signal'].rolling(window=window).std()
    train["rolling_var_" + str(window)] = train['signal'].rolling(window=window).var()
    train["rolling_min_" + str(window)] = train['signal'].rolling(window=window).min()
    train["rolling_max_" + str(window)] = train['signal'].rolling(window=window).max()

train = train.replace([np.inf, -np.inf], np.nan)    
train.fillna(0, inplace=True)

CPU times: user 3.21 s, sys: 222 ms, total: 3.43 s
Wall time: 3.44 s


In [18]:
%%time
def features(df):
    df = df.sort_values(by=['time']).reset_index(drop=True)
    df.index = ((df.time * 10_000) - 1).values
    df['batch'] = df.index // 25_000
    df['batch_index'] = df.index  - (df.batch * 25_000)
    df['batch_slices'] = df['batch_index']  // 2500
    df['batch_slices2'] = df.apply(lambda r: '_'.join([str(r['batch']).zfill(3), str(r['batch_slices']).zfill(3)]), axis=1)
    
    for c in ['batch','batch_slices2']:
        d = {}
        d['mean'+c] = df.groupby([c])['signal'].mean()
        d['median'+c] = df.groupby([c])['signal'].median()
        d['max'+c] = df.groupby([c])['signal'].max()
        d['min'+c] = df.groupby([c])['signal'].min()
        d['std'+c] = df.groupby([c])['signal'].std()
        d['mean_abs_chg'+c] = df.groupby([c])['signal'].apply(lambda x: np.mean(np.abs(np.diff(x))))
        d['abs_max'+c] = df.groupby([c])['signal'].apply(lambda x: np.max(np.abs(x)))
        d['abs_min'+c] = df.groupby([c])['signal'].apply(lambda x: np.min(np.abs(x)))
        d['range'+c] = d['max'+c] - d['min'+c]
        d['maxtomin'+c] = d['max'+c] / d['min'+c]
        d['abs_avg'+c] = (d['abs_min'+c] + d['abs_max'+c]) / 2
        for v in d:
            df[v] = df[c].map(d[v].to_dict())
    # add shifts_1
    df['signal_shift_+1'] = [0,] + list(df['signal'].values[:-1])
    df['signal_shift_-1'] = list(df['signal'].values[1:]) + [0]
    for i in df[df['batch_index']==0].index:
        df['signal_shift_+1'][i] = np.nan
    for i in df[df['batch_index']==49999].index:
        df['signal_shift_-1'][i] = np.nan
    
    # add shifts_2 - my upgrade
    df['signal_shift_+2'] = [0,] + [1,] + list(df['signal'].values[:-2])
    df['signal_shift_-2'] = list(df['signal'].values[2:]) + [0] + [1]
    for i in df[df['batch_index']==0].index:
        df['signal_shift_+2'][i] = np.nan
    for i in df[df['batch_index']==1].index:
        df['signal_shift_+2'][i] = np.nan
    for i in df[df['batch_index']==49999].index:
        df['signal_shift_-2'][i] = np.nan
    for i in df[df['batch_index']==49998].index:
        df['signal_shift_-2'][i] = np.nan
    
    df = df.drop(columns=['batch', 'batch_index', 'batch_slices', 'batch_slices2'])
    
    for c in [c1 for c1 in df.columns if c1 not in ['time', 'signal', 'open_channels']]:
        df[c+'_msignal'] = df[c] - df['signal']
        
    df = df.replace([np.inf, -np.inf], np.nan)    
    df.fillna(0, inplace=True)
    gc.collect()
    return df

train = features(train)
test = features(test)
    

CPU times: user 2min 34s, sys: 3.52 s, total: 2min 38s
Wall time: 2min 37s


In [19]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to 777.24 Mb (73.2% reduction)
Mem. usage decreased to 308.99 Mb (73.0% reduction)


In [0]:
y = train['open_channels']
col = [c for c in train.columns if c not in ['time', 'open_channels', 'group']]

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 5000000 entries, 0.0 to 4999999.0
Data columns (total 75 columns):
time                                 float64
signal                               float16
open_channels                        int8
rolling_mean_10                      float16
rolling_std_10                       float16
rolling_var_10                       float16
rolling_min_10                       float16
rolling_max_10                       float16
rolling_mean_50                      float16
rolling_std_50                       float16
rolling_var_50                       float16
rolling_min_50                       float16
rolling_max_50                       float16
meanbatch                            float16
medianbatch                          float16
maxbatch                             float16
minbatch                             float16
stdbatch                             float16
mean_abs_chgbatch                    float16
abs_maxbatch                 

In [22]:
train.head()

,time,signal,open_channels,rolling_mean_10,rolling_std_10,rolling_var_10,rolling_min_10,rolling_max_10,rolling_mean_50,rolling_std_50,rolling_var_50,rolling_min_50,rolling_max_50,meanbatch,medianbatch,maxbatch,minbatch,stdbatch,mean_abs_chgbatch,abs_maxbatch,abs_minbatch,rangebatch,maxtominbatch,abs_avgbatch,meanbatch_slices2,medianbatch_slices2,maxbatch_slices2,minbatch_slices2,stdbatch_slices2,mean_abs_chgbatch_slices2,abs_maxbatch_slices2,abs_minbatch_slices2,rangebatch_slices2,maxtominbatch_slices2,abs_avgbatch_slices2,signal_shift_+1,signal_shift_-1,signal_shift_+2,signal_shift_-2,rolling_mean_10_msignal,rolling_std_10_msignal,rolling_var_10_msignal,rolling_min_10_msignal,rolling_max_10_msignal,rolling_mean_50_msignal,rolling_std_50_msignal,rolling_var_50_msignal,rolling_min_50_msignal,rolling_max_50_msignal,meanbatch_msignal,medianbatch_msignal,maxbatch_msignal,minbatch_msignal,stdbatch_msignal,mean_abs_chgbatch_msignal,abs_maxbatch_msignal,abs_minbatch_msignal,rangebatch_msignal,maxtominbatch_msignal,abs_avgbatch_msignal,meanbatch_slices2_msignal,medianbatch_slices2_msignal,maxbatch_slices2_msignal,minbatch_slices2_msignal,stdbatch_slices2_msignal,mean_abs_chgbatch_slices2_msignal,abs_maxbatch_slices2_msignal,abs_minbatch_slices2_msignal,rangebatch_slices2_msignal,maxtominbatch_slices2_msignal,abs_avgbatch_slices2_msignal,signal_shift_+1_msignal,signal_shift_-1_msignal,signal_shift_+2_msignal,signal_shift_-2_msignal
0.0,0.0001,-2.759766,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.683594,-2.691406,-0.765137,-3.654297,0.272949,0.27417,3.654297,0.765137,2.890625,0.209351,2.210938,-2.693359,-2.693359,-1.90332,-3.466797,0.244385,0.269043,3.466797,1.90332,1.564453,0.548828,2.685547,0.000000,-2.855469,0.000000,-2.408203,2.759766,2.759766,2.759766,2.759766,2.759766,2.759766,2.759766,2.759766,2.759766,2.759766,0.076233,0.067688,1.995117,-0.895020,3.033203,3.035156,6.414062,3.525391,5.648438,2.968750,4.968750,0.066223,0.065979,0.856934,-0.707031,3.003906,3.029297,6.226562,4.664062,4.324219,3.308594,5.445312,0.000000,-0.095703,0.000000,0.352539
1.0,0.0002,-2.855469,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.683594,-2.691406,-0.765137,-3.654297,0.272949,0.27417,3.654297,0.765137,2.890625,0.209351,2.210938,-2.693359,-2.693359,-1.90332,-3.466797,0.244385,0.269043,3.466797,1.90332,1.564453,0.548828,2.685547,-2.759766,-2.408203,0.000000,-3.140625,2.855469,2.855469,2.855469,2.855469,2.855469,2.855469,2.855469,2.855469,2.855469,2.855469,0.171997,0.163452,2.089844,-0.799316,3.128906,3.130859,6.511719,3.621094,5.746094,3.064453,5.066406,0.161987,0.161743,0.952637,-0.611328,3.099609,3.125000,6.324219,4.757812,4.417969,3.404297,5.539062,0.095703,0.448242,0.000000,-0.284668
2.0,0.0003,-2.408203,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.683594,-2.691406,-0.765137,-3.654297,0.272949,0.27417,3.654297,0.765137,2.890625,0.209351,2.210938,-2.693359,-2.693359,-1.90332,-3.466797,0.244385,0.269043,3.466797,1.90332,1.564453,0.548828,2.685547,-2.855469,-3.140625,-2.759766,-3.152344,2.408203,2.408203,2.408203,2.408203,2.408203,2.408203,2.408203,2.408203,2.408203,2.408203,-0.276367,-0.284912,1.642578,-1.248047,2.679688,2.681641,6.062500,3.171875,5.296875,2.617188,4.617188,-0.286377,-0.286621,0.504395,-1.059570,2.652344,2.675781,5.875000,4.308594,3.970703,2.957031,5.093750,-0.448242,-0.732910,-0.352539,-0.745117
3.0,0.0004,-3.140625,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.683594,-2.691406,-0.765137,-3.654297,0.272949,0.27417,3.654297,0.765137,2.890625,0.209351,2.210938,-2.693359,-2.693359,-1.90332,-3.466797,0.244385,0.269043,3.466797,1.90332,1.564453,0.548828,2.685547,-2.408203,-3.152344,-2.855469,-2.642578,3.140625,3.140625,3.140625,3.140625,3.140625,3.140625,3.140625,3.140625,3.140625,3.140625,0.456543,0.447998,2.375000,-0.514648,3.414062,3.414062,6.796875,3.906250,6.031250,3.349609,5.351562,0.446533,0.446289,1.237305,-0.326416,3.384766,3.410156,6.605469,5.042969,4.703125,3.689453,5.824219,0.732910,-0.012100,0.284668,0.498535
4.0,0.0005,-3.152

In [23]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 2000000 entries, 5000000.0 to 6999999.0
Data columns (total 74 columns):
time                                 float64
signal                               float16
rolling_mean_10                      float16
rolling_std_10                       float16
rolling_var_10                       float16
rolling_min_10                       float16
rolling_max_10                       float16
rolling_mean_50                      float16
rolling_std_50                       float16
rolling_var_50                       float16
rolling_min_50                       float16
rolling_max_50                       float16
meanbatch                            float16
medianbatch                          float16
maxbatch                             float16
minbatch                             float16
stdbatch                             float16
mean_abs_chgbatch                    float16
abs_maxbatch                         float16
abs_minbatch        

In [24]:
test.head()

,time,signal,rolling_mean_10,rolling_std_10,rolling_var_10,rolling_min_10,rolling_max_10,rolling_mean_50,rolling_std_50,rolling_var_50,rolling_min_50,rolling_max_50,meanbatch,medianbatch,maxbatch,minbatch,stdbatch,mean_abs_chgbatch,abs_maxbatch,abs_minbatch,rangebatch,maxtominbatch,abs_avgbatch,meanbatch_slices2,medianbatch_slices2,maxbatch_slices2,minbatch_slices2,stdbatch_slices2,mean_abs_chgbatch_slices2,abs_maxbatch_slices2,abs_minbatch_slices2,rangebatch_slices2,maxtominbatch_slices2,abs_avgbatch_slices2,signal_shift_+1,signal_shift_-1,signal_shift_+2,signal_shift_-2,rolling_mean_10_msignal,rolling_std_10_msignal,rolling_var_10_msignal,rolling_min_10_msignal,rolling_max_10_msignal,rolling_mean_50_msignal,rolling_std_50_msignal,rolling_var_50_msignal,rolling_min_50_msignal,rolling_max_50_msignal,meanbatch_msignal,medianbatch_msignal,maxbatch_msignal,minbatch_msignal,stdbatch_msignal,mean_abs_chgbatch_msignal,abs_maxbatch_msignal,abs_minbatch_msignal,rangebatch_msignal,maxtominbatch_msignal,abs_avgbatch_msignal,meanbatch_slices2_msignal,medianbatch_slices2_msignal,maxbatch_slices2_msignal,minbatch_slices2_msignal,stdbatch_slices2_msignal,mean_abs_chgbatch_slices2_msignal,abs_maxbatch_slices2_msignal,abs_minbatch_slices2_msignal,rangebatch_slices2_msignal,maxtominbatch_slices2_msignal,abs_avgbatch_slices2_msignal,signal_shift_+1_msignal,signal_shift_-1_msignal,signal_shift_+2_msignal,signal_shift_-2_msignal
5000000.0,500.0001,-2.650391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.269531,-2.285156,-0.570312,-3.605469,0.376221,0.277344,3.605469,0.570312,3.035156,0.158081,2.087891,-2.654297,-2.654297,-1.794922,-3.398438,0.244385,0.266846,3.398438,1.794922,1.603516,0.52832,2.597656,0.000000,-2.849609,0.000000,-2.859375,2.650391,2.650391,2.650391,2.650391,2.650391,2.650391,2.650391,2.650391,2.650391,2.650391,0.380371,0.364258,2.080078,-0.956543,3.025391,2.927734,6.257812,3.220703,5.687500,2.808594,4.738281,-0.004524,-0.004551,0.854492,-0.749023,2.894531,2.916016,6.046875,4.445312,4.253906,3.177734,5.246094,0.000000,-0.199585,0.000000,-0.210205
5000001.0,500.0002,-2.849609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.269531,-2.285156,-0.570312,-3.605469,0.376221,0.277344,3.605469,0.570312,3.035156,0.158081,2.087891,-2.654297,-2.654297,-1.794922,-3.398438,0.244385,0.266846,3.398438,1.794922,1.603516,0.52832,2.597656,-2.650391,-2.859375,0.000000,-2.435547,2.849609,2.849609,2.849609,2.849609,2.849609,2.849609,2.849609,2.849609,2.849609,2.849609,0.580078,0.563965,2.279297,-0.756836,3.226562,3.126953,6.457031,3.419922,5.886719,3.007812,4.937500,0.195068,0.195068,1.054688,-0.549316,3.093750,3.117188,6.250000,4.644531,4.453125,3.376953,5.445312,0.199585,-0.010597,0.000000,0.414307
5000002.0,500.0003,-2.859375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.269531,-2.285156,-0.570312,-3.605469,0.376221,0.277344,3.605469,0.570312,3.035156,0.158081,2.087891,-2.654297,-2.654297,-1.794922,-3.398438,0.244385,0.266846,3.398438,1.794922,1.603516,0.52832,2.597656,-2.849609,-2.435547,-2.650391,-2.615234,2.859375,2.859375,2.859375,2.859375,2.859375,2.859375,2.859375,2.859375,2.859375,2.859375,0.590820,0.574219,2.289062,-0.746094,3.236328,3.136719,6.464844,3.429688,5.894531,3.017578,4.949219,0.205688,0.205688,1.064453,-0.538574,3.103516,3.126953,6.257812,4.656250,4.464844,3.388672,5.457031,0.010597,0.425049,0.210205,0.244507
5000003.0,500.0004,-2.435547,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.269531,-2.285156,-0.570312,-3.605469,0.376221,0.277344,3.605469,0.570312,3.035156,0.158081,2.087891,-2.654297,-2.654297,-1.794922,-3.398438,0.244385,0.266846,3.398438,1.794922,1.603516,0.52832,2.597656,-2.859375,-2.615234,-2.849609,-2.566406,2.435547,2.435547,2.435547,2.435547,2.435547,2.435547,2.435547,2.435547,2.435547,2.435547,0.165649,0.149292,1.865234,-1.170898,2.810547,2.712891,6.042969,3.005859,5.472656,2.593750,4.523438,-0.219360,-0.219360,0.640137,-0.963867,2.679688,2.701172,5.832031,4.230469,4.039062,2.962891,5.031250,-0.425049,-0.180542,-0.414307,-0.130859

In [0]:
train.to_pickle('gdrive/My Drive/Colab Notebooks/Kaggle/Liverpool_Ion_Switching/train1.pkl')
test.to_pickle('gdrive/My Drive/Colab Notebooks/Kaggle/Liverpool_Ion_Switching/test1.pkl')